<a href="https://colab.research.google.com/github/min207/2023-1-ESAA/blob/main/ESAA230519.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **08. 파이썬 추천 시스템 패키지 - Surprise**

----
### **| Surprise 패키지 소개**
- Surprise: 추천 시스템 구축을 위한 전용 패키지
  + conda나 pip를 통해 설치 
  + 다양한 추천 알고리즘, 예를 들어 사용자 도는 아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용해 추천 시스템을 구축할 수 있음
  + Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성됨

In [1]:
! pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095447 sha256=0e1625eb0c4661ba5d71229b4e01722d44299ffd20b366c5bc1d030a9527a245
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


----
### **| Surprise를 이용한 추천 시스템 구축**

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

> - 추천을 위한 데이터 세트 로딩
- Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능
- Surprise의 무비렌즈 사이트에서 제공하는 과거 버전의 데이터 세트 불러오는 API 제공 

In [3]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] ㅛ
Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


> - 과거버전의 데이터 세트로 분리 문자기가 탭 문자
- 다운 받은 데이터 파일과 동일하게 로우 레벨의 사용자-아이템 평점 데이터를 그대로 적용해야함




- SVD로 잠재 요인 협업 필터링 수행 

In [4]:
algo = SVD() 
algo.fit(trainset)

> - 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천 수행
- 추천 예측 메서드는 test(), predict()
  + test(): 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드
  + predict(): 개별 사용자와 영화에 대한 추천 평점 반환

In [5]:
predictions = algo.test(testset)
print('predictions type :', type(predictions), 'size :', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

predictions type : <class 'list'> size : 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4830043802523236, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=4.196325635676174, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9268591933515173, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.5661004500456523, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.4821302192833867, details={'was_impossible': False})]

> - SVD 알고리즘 객체의 test 메서드의 호출 결과는 파이썬 리스트이며 크기는 입력 인자 데이터 세트의 크기와 동일
- 내부에 Prediction 객체를 가짐
  + Prediction: Surprise 패키지에서 제공하는 데이터 타입, 추천 예측 평점 데이터를 튜플 형태로 가짐
  + 로그용으로 데이터를 남기는 데 사용됨
  + 'was_impossible'이 True이면 예측값을 생성할 수 없는 데이터라는 의미

In [6]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.4830043802523236),
 ('882', '291', 4.196325635676174),
 ('535', '507', 3.9268591933515173)]

> - predict를 이용해 추천 예측 수행: 개별 사용자의 아이템에 대한 추천 평점 예측
- 사용자 아이디, 아이템 아이디를 문자열로 입력 

In [7]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.26   {'was_impossible': False}


> - 추천 예측 평점을 est로 반환
- 즉, test 메서드는 predict를 반복적으로 수행한 결과



- Surprise의 accuracy 모듈은 RMSE, MSE 등의 방법으로 추천 시스테의 성능 평가 정보 제공 

In [8]:
accuracy.rmse(predictions)

RMSE: 0.9484


0.9483604706690945

### **| Surprise 주요 모듈 소개**
- Dataset
  + user_id, item_id, rating 데이터가 로우 레벨로 된 데이터 세트만 적용 가능
  + 칼럼 순서가 반드시 사용자 아이디, 아이템 아이디, 평점 순이어야 함

- OS 파일 데이터를 Surpirse 데이터 세트로 로딩
  + Dataset.load_from_file API를 이용해 지정된 디렉터리에 있는 사용자-아이템 평점 데이터 로딩
  + 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있으면 안됨 

In [9]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')
# ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('ratings_noh.csv', index=False, header=False)

> - 헤더가 삭제된 새로운 파일 생성
 - 각 칼럼이 헤더 없이 콤마로만 분리되어 있음: reader 클래스의 생성자에 각 필드의 칼럼명과 칼럼 분리문자, 최소~최대 평점을 입력해 객체 생성 후 데이터 로딩 

In [11]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('ratings_noh.csv', reader=reader)

> - Reader 클래스의 주요 생성 파라미터
    + line_format(string)
    + sep(char)
    + rating_scale(tuple, optional)



- SVD 행렬 분해 기법을 이용해 추천 예측 수행
  + 잠재 요인 크기 K 값을 나타내는 파라미터인 n_factors를 50으로 설정하여 학습 후 테스트 데이터 세트 적용
  + 그 후 예측/실제 평점 데이터를 RMSE로 평가 

In [12]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

- 판다스 DataFrame에서 Surprise 데이터 세트로 로딩 
  + Dataset.load_from_df()를 이용
  + 마찬가지로 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야함 

In [13]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### **| Surprise 추천 알고리즘 클래스**
- 추천 예측을 위해 자주 사용되는 추천 알고리즘 클래스
  + SVD
  + KNNBasic
  + BaselineOnly

- Surprise SVD의 비용함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization을 적용한 것 

- SVD 클래스의 입력 파라미터
  + n_factors
  + n_epochs
  + biasd(bool)

- SVD++ 알고리즘의 RMSE, MAE 성적이 가장 좋지만 상대적으로 시간이 너무 오래 걸려 데이터가 조금만 더 커져도 사용하기 어려움
- SVD와 k-NN Baseline의 성능 평가 수치가 가장 좋음
  + Baseline: 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식 

### **| 베이스라인 평점**
- 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것
  + 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수

### **| 교차 검증과 하이퍼 파라미터 튜닝**
- Surprise는 교차 검증과 하이퍼 파라미터 튜닝을 위해 사이킷런과 유사한 cross_validate()와 GridSearchCV 클래스 제공
  + surprise.model_selection 모듈 내에 존재하며, 폴드된 데이터 세트의 개수와 성능 측정 방법을 명시해 교차 검증 수행 
  + 인자로 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 세트 개수 입력 

In [14]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8708  0.8707  0.8745  0.8773  0.8759  0.8738  0.0027  
MAE (testset)     0.6707  0.6693  0.6722  0.6715  0.6725  0.6712  0.0012  
Fit time          1.34    1.40    2.20    2.05    3.37    2.07    0.74    
Test time         0.12    0.27    0.20    0.22    0.25    0.21    0.05    


{'test_rmse': array([0.87080597, 0.87065633, 0.87447292, 0.87730159, 0.87589612]),
 'test_mae': array([0.67073441, 0.66926881, 0.67216531, 0.67150453, 0.67251571]),
 'fit_time': (1.335892677307129,
  1.3974175453186035,
  2.200138568878174,
  2.0464694499969482,
  3.3711040019989014),
 'test_time': (0.12497925758361816,
  0.271726131439209,
  0.20223331451416016,
  0.21807456016540527,
  0.24524784088134766)}

> - 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 평가 수치를 함께 보여줌


- GridSearchCV도 교차 검증을 통한 하이퍼 파라미터 최적화 수행
- n_epochs와 SVD 잠재요인 K의 크기를 지정하는 n_factors 튜닝 

In [15]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20, 40, 60],
            'n_factors':[50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가를 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8767188271152314
{'n_epochs': 20, 'n_factors': 50}


### **| Surprise를 이용한 개인화 영화 추천 시스템 구축**
- 학습/테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용
  + 그러나 TrainSet 클래스 객체로 변환하지 않으면 fit()을 통해 학습 불가
  + DatasetAutoFolds 클래스를 이용해 해결 

In [17]:
# 분리하지 않고 데이터 사용시 오류

#data=Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#algo=SVD(n_factors=50, random_state=0)
#algo.fit(data)

In [18]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성
data_folds = DatasetAutoFolds(ratings_file='ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함
trainset = data_folds.build_full_trainset()

> - SVD로 학습 수행 

In [19]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

> - 특정 사용자는 userId = 9인 사용자로 지정

In [20]:
# 영화에 대한 상세 속성 정보 DataFrame 로딩
movies = pd.read_csv('movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


> - predict() 메서드를 이용해 userId9 사용자의 추천 예상 평점 계산
    + userId, movieId 값 입력

In [21]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


> - 추천 예측 평점: 3.13



- get_unseen_surprise() 함수를 생성해 아직 평점을 매기지 않은 영화 정보 반환 

In [22]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


- SVD를 이용해 높은 예측 평점을 가진 순으로 영화 추천
  + recomm_movie_by_surprise() 함수 생성

In [23]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list로 저장
  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # predictions list 객체는 surprise의 predictions 객체를 원소로 갖고 있음
  # [Prediction(uid='9', iid='1', est=3.69), Predictions(uid='9', iid='2', est=2.98),,,]

  # 이를 est 값으로 정렬하기 위해 아리애 sortkey_est 함수 정의함
  # sortdey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림차순으로 정렬 수행하고 top_n개의 최상위 값 추출
  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  # top_n 으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
  top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('### top-10 추천 영화 리스트 ###')
for top_movie in top_movie_preds:
  print(top_movie[1], ':', top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
### top-10 추천 영화 리스트 ###
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


> - 세븐, 유주얼 서스펙트, 대부, 좋은 친구들 같은 서스펜드/스릴러/범죄 영화 및 스타워즈 같은 액션 영화 위주로 추천됨 